Required imports

In [3]:
import h5py
import pandas as pd
import numpy as np

import torch.nn as nn
import torch
from torch.utils.data import Subset, Dataset, DataLoader
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR

from tqdm import tqdm
from numpy import vstack
from numpy import argmax

from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, median_absolute_error, r2_score
from scipy.stats import median_absolute_deviation



Data overview

In [4]:
data_pd = pd.read_hdf("SP500_Options_Monthly.h5")

In [5]:
data_pd

,optid,date,secid,cp_flag,strike,bid,ask,volume,openint,impvol,...,vix,dhedged_return_mid,dhedged_return_spot,dhedged_return_spot_gamma,dhedged_return_mid_delev,dhedged_return_spot_delev,dhedged_return_spot_gamma_delev,IV_mness_deriv_1,IV_ttm_deriv_1,short_rate
0,10003234,1996-12-20,108105,P,825.0,79.750,80.750,4.0,4.0,0.163276,...,18.85,0.004046,0.000434,0.191613,0.000882,9.452300e-05,0.041769,NaN,NaN,0.052017
1,10003234,1997-01-17,108105,P,825.0,62.250,64.250,5.0,119.0,0.166865,...,18.63,0.056713,0.004622,2.198635,0.011167,9.099802e-04,0.432912,-0.030072,0.000032,0.054244
2,10003234,1997-02-21,108105,P,825.0,56.625,58.625,17.0,94.0,0.176886,...,20.55,-0.029893,-0.002148,-1.116086,-0.005773,-4.149493e-04,-0.215558,-0.036234,0.000013,0.054835
3,10003234,1997-03-21,108105,P,825.0,62.375,64.375,0.0,1238.0,0.182198,...,19.69,-0.081126,-0.006557,-3.291678,-0.016392,-1.324868e-03,-0.665094,-0.034897,0.000010,0.060077
4,10003234,1997-04-18,108105,P,825.0,64.875,66.875,300.0,1250.0,0.175205,...,18.75,0.092304,0.007934,3.583767,0.018166,1.561578e-03,0.705320,-0.028753,0.000017,0.056968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180809,117577568,2017-10-20,108105,P,2695.0,121.300,127.100,0.0,19.0,0.125499,...,9.97,-0.078302,-0.003776,-2.001284,-0.004168,-2.010069e-04,-0.106522,0.037274,0.000060,0.012243
180810,117595317,2017-10-20,108105,C,2735.0,0.150,0.300,0.0,162.0,0.095167,...,9.97,-0.955706,-0.000084,-0.208233,-0.008319,-7.268758e-07,-0.001813,0.004745,0.000174,0.012243
180811,117601432,2017-10-20,108105,C,2665.0,2.450,2.700,39.0,143.0,0.065220,...,9.97,-0.365931,-0.000366,-0.151199,-0.004222,-4.221724e-06,-0.001745,0.002268,0.000167,0.012243
180812,117646963,2017-10-20,108105,C,2770.0,0.550,0.700,0.0,10.0,0.090533,...,9.97,-0.553955,-0.000134,-0.261057,-0.007028,-1.705668e-06,-0.003312,0.002268,0.000167,0.012243


In [6]:
features = ['mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta', 'strike', 'delta', 'spot_close'
                   , 'divrate', 'vix', 'short_rate', 'midprice', 'date']

In [8]:
put_df = data_pd[data_pd['cp_flag'] == 'P'][features]
call_df = data_pd[data_pd['cp_flag'] == 'C'][features]
put_df

,mness,ttm,embed_lev,impvol,gamma,vega,theta,strike,delta,spot_close,divrate,vix,short_rate,midprice,date
0,0.493527,728,4.587453,0.163276,0.002263,392.61790,2.388857,825.0,-0.499509,748.87,0.021764,18.85,0.052017,80.250,1996-12-20
1,0.264026,700,5.078717,0.166865,0.002102,404.58860,-1.370254,825.0,-0.413864,776.17,0.022156,18.63,0.054244,63.250,1997-01-17
2,0.119626,665,5.177660,0.176886,0.001925,398.23850,-4.685850,825.0,-0.372130,801.77,0.022782,20.55,0.054835,57.625,1997-02-21
3,0.211250,637,4.949190,0.182198,0.001992,388.80880,-3.081652,825.0,-0.400019,784.10,0.023084,19.69,0.060077,63.375,1997-03-21
4,0.325910,609,5.081052,0.175205,0.002214,379.39500,-0.389018,825.0,-0.436770,766.34,0.023036,18.75,0.056968,65.875,1997-04-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180803,-0.625458,56,39.826654,0.104692,0.003099,324.23790,-114.568500,2510.0,-0.260592,2575.21,0.018232,9.97,0.012243,16.850,2017-10-20
180804,-0.467571,56,40.341208,0.096709,0.003673,354.91720,-116.236200,2530.0,-0.315654,2575.21,0.018232,9.97,0.012243,20.150,2017-10-20
180805,0.255425,56,36.660494,0.076517,0.005031,384.66660,-102.074900,2595.0,-0.600756,2575.21,0.018232,9.97,0.012243,42.200,2017-10-20
180809,1.308051,28,18.787525,0.125499,0.001887,116.19330,-106.506800,2695.0,-0.906105,2575.21,0.018232,9.97,0.012243,124.200,2017-10-20


Forming dataset with additional features (differeting between call and put options, required for task 8).

In [9]:
put_plus = put_df
call_plus = call_df
for c in put_df.columns:
    if c!='midprice' and c != 'date':
        call_plus[c+"_additional"] = np.zeros(len(call_plus[c]))
        put_plus[c+"_additional"] = put_df[c]
put_plus

,mness,ttm,embed_lev,impvol,gamma,vega,theta,strike,delta,spot_close,...,impvol_additional,gamma_additional,vega_additional,theta_additional,strike_additional,delta_additional,spot_close_additional,divrate_additional,vix_additional,short_rate_additional
0,0.493527,728,4.587453,0.163276,0.002263,392.61790,2.388857,825.0,-0.499509,748.87,...,0.163276,0.002263,392.61790,2.388857,825.0,-0.499509,748.87,0.021764,18.85,0.052017
1,0.264026,700,5.078717,0.166865,0.002102,404.58860,-1.370254,825.0,-0.413864,776.17,...,0.166865,0.002102,404.58860,-1.370254,825.0,-0.413864,776.17,0.022156,18.63,0.054244
2,0.119626,665,5.177660,0.176886,0.001925,398.23850,-4.685850,825.0,-0.372130,801.77,...,0.176886,0.001925,398.23850,-4.685850,825.0,-0.372130,801.77,0.022782,20.55,0.054835
3,0.211250,637,4.949190,0.182198,0.001992,388.80880,-3.081652,825.0,-0.400019,784.10,...,0.182198,0.001992,388.80880,-3.081652,825.0,-0.400019,784.10,0.023084,19.69,0.060077
4,0.325910,609,5.081052,0.175205,0.002214,379.39500,-0.389018,825.0,-0.436770,766.34,...,0.175205,0.002214,379.39500,-0.389018,825.0,-0.436770,766.34,0.023036,18.75,0.056968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180803,-0.625458,56,39.826654,0.104692,0.003099,324.23790,-114.568500,2510.0,-0.260592,2575.21,...,0.104692,0.003099,324.23790,-114.568500,2510.0,-0.260592,2575.21,0.018232,9.97,0.012243
180804,-0.467571,56,40.341208,0.096709,0.003673,354.91720,-116.236200,2530.0,-0.315654,2575.21,...,0.096709,0.003673,354.91720,-116.236200,2530.0,-0.315654,2575.21,0.018232,9.97,0.012243
180805,0.255425,56,36.660494,0.076517,0.005031,384.66660,-102.074900,2595.0,-0.600756,2575.21,...,0.076517,0.005031,384.66660,-102.074900,2595.0,-0.600756,2575.21,0.018232,9.97,0.012243
180809,1.308051,28,18.787525,0.125499,0.001887,116.19330,-106.506800,2695.0,-0.906105,2575.21,...,0.125499,0.001887,116.19330,-106.506800,2695.0,-0.906105,2575.21,0.018232,9.97,0.012243


In [10]:
call_plus

,mness,ttm,embed_lev,impvol,gamma,vega,theta,strike,delta,spot_close,...,impvol_additional,gamma_additional,vega_additional,theta_additional,strike_additional,delta_additional,spot_close_additional,divrate_additional,vix_additional,short_rate_additional
42,1.857294,28,55.633217,0.169906,0.002070,16.72431,-20.102380,875.0,0.034694,801.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,0.488979,56,30.652784,0.132271,0.011247,92.88889,-52.013510,685.0,0.355372,666.84,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,1.874161,28,70.045687,0.134731,0.003244,13.18954,-12.747660,685.0,0.034270,638.73,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,2.210994,119,29.281115,0.166849,0.000412,39.72491,-11.555700,1650.0,0.021907,1336.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,2.612415,91,38.801964,0.162164,0.000187,13.34660,-4.829397,1650.0,0.007264,1335.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180807,2.417067,35,115.343944,0.087773,0.000288,14.71647,-6.823027,2670.0,0.006920,2500.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180808,1.998424,28,133.106167,0.075428,0.000942,34.87271,-17.557830,2685.0,0.020675,2575.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180810,2.283916,28,114.877257,0.095167,0.000401,18.72413,-11.936470,2735.0,0.010037,2575.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180811,1.341601,56,86.671068,0.065220,0.002420,157.72820,-33.382420,2665.0,0.086664,2575.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
total_df = pd.concat([call_plus, put_plus], axis=0)
total_df.sort_values(by='date')
total_df = total_df.reset_index()
total_df.to_csv("total_df.csv", index=False)

In [12]:
min(data_pd['midprice']), max(data_pd['midprice']), np.mean(data_pd['midprice'])

(0.025, 2212.45, 79.14769801287511)

Defining required classes

In [128]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.layer0 = nn.Linear(13, 20)
        self.layer1 = nn.Linear(20, 32)
        self.layer2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.layer0(x)
        x = self.act(x)
        x = self.layer1(x)
        x = self.act(x)
        x = self.layer2(x)
        x = self.act(x)
        x = self.out(x)
        return x

class NeuralNetExtended(nn.Module):
    def __init__(self):
        super(NeuralNetExtended, self).__init__()

        self.layer0 = nn.Linear(26, 20)
        self.layer1 = nn.Linear(20, 32)
        self.layer2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
        self.act = nn.Sigmoid()

    def forward(self, x):
        x = self.layer0(x)
        x = self.act(x)
        x = self.layer1(x)
        x = self.act(x)
        x = self.layer2(x)
        x = self.act(x)
        x = self.out(x)
        return x



In [111]:

# https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=3, verbose=False, delta=0.001, path='checkpoint.pt', trace_func=print):

        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            
        elif score >= self.best_score - self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        elif score < self.best_score - self.delta:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.8f} --> {val_loss:.8f}).  Saving model ...\n', val_loss - self.val_loss_min)
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [119]:


class Config:

    def __init__(self):

        # related to data
        self.path_to_dataset =  "/home/sgarkot/code/options/SP500_Options_Monthly.h5"

        # related to model
        self.num_epochs = 5
        
        self.device = torch.device("cpu")
#         self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#         self.criterion = nn.L1Loss()
        self.criterion = nn.MSELoss()

    def path(self, fold, extended=False):
        if extended:
            return "/home/sgarkot/code/options/DNN/model_extended/fold" + str(fold)
        return  "/home/sgarkot/code/options/DNN/model/fold_" + str(fold)

class OptionsDataset(Dataset):

    # load the dataset
    def __init__(self, path, features, csv=False):

        # load the csv file as a dataframe
        if csv:
            df = pd.read_csv(path).sort_values(by='date')
        else:
            df = pd.read_hdf(path).sort_values(by='date')
        df = df.reset_index()
#         df = df[df['midprice']<1000]
        
        self.df = df 
        # store the inputs and outputs
        self.X = df[features].to_numpy()
        self.y = pd.DataFrame({'response': df['midprice']}).to_numpy()

        self.X = torch.tensor(self.X)
        self.y = torch.tensor(self.y)

        # convert to needed further types
        self.X = self.X.long()
        self.y = self.y.to(torch.float64)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def get_splits(self, fold):
        train_pc = 0.85

        date = "2007-01-"
        if fold == 1:
            date = "2012-01-"
        elif fold == 2:
            date = "2017-01-"

        series = self.df.date.apply(lambda x: str(x)).str.startswith(date)
        frame = {f"is_January_{date[:5]}": series}
        df = pd.DataFrame(frame)
        january_df = df.loc[df[f'is_January_{date[:5]}']==True]

        last_date_ind = int(list(january_df.index)[-1])
        
        train = [i for i in range(0, int(last_date_ind * train_pc))]
        test = [i for i in range(int(last_date_ind * train_pc), last_date_ind + 1)]

        return (train, test)



Defining required helping functions

In [136]:
from sklearn.metrics import mean_squared_error, r2_score
def run_train_epoch(cfg, model, train_dl, criterion, optimizer):

    avg_loss = 0

    for i, (inputs, targets) in tqdm(enumerate(train_dl)):
        
        inputs = inputs.to(torch.float)

        targets = targets.to(torch.float)
        
        inputs = inputs.to(cfg.device)
        targets = targets.to(cfg.device)

        # clear the gradients
        optimizer.zero_grad()

        # compute the model output
        yhat = model(inputs)

        # calculate loss
        loss = criterion(yhat, targets)

        avg_loss += loss

        # credit assignment
        loss.backward()

        # update model weights
        optimizer.step()

    avg_loss = avg_loss / len(train_dl.dataset)
    

    return  avg_loss

def run_validation_epoch(cfg, model, test_dl, criterion, return_results=False):

    results = list()

    predictions, actuals = list(), list()

    loss = 0
    d_y = test_dl.dataset.dataset.y

    for i, (inputs, targets) in tqdm(enumerate(test_dl)):

        inputs = inputs.to(torch.float)
        
        targets = targets.to(torch.float)
        targets = targets.to(cfg.device)
        inputs = inputs.to(cfg.device)

        yhat = model(inputs)

        loss += criterion(yhat, targets)

        # retrieve numpy array
        yhat = yhat.detach().cpu().numpy()

        yhat = np.asarray(yhat)
        actual = targets.detach().cpu().numpy()

        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(actual), 1))
        
        if return_results:
            for predicted_value, actual_value in zip(yhat, actual):
                results.append(
                    [float(actual_value), float(predicted_value)])

        
        # storing
        predictions.append(yhat)
        actuals.append(actual)
        
    if return_results:
        summary_df = pd.DataFrame(results, columns=["label", "predicted"])
        return summary_df, loss / len(test_dl.dataset)
    
    loss = loss / len(test_dl.dataset)
    return loss

def train_for_a_fold(cfg, fold, dataset, extended=False):

    PATH_TO_SAVE = cfg.path(fold, extended= extended)
    if extended:
        model = NeuralNetExtended()
    else:
        model = NeuralNet()
    model.to(cfg.device)

    criterion = cfg.criterion

    optimizer = SGD(model.parameters(), lr=0.1)

    stopper = EarlyStopping(patience=10, verbose=True, path = PATH_TO_SAVE + '/fold' + str(fold) + '_regression.pth')

    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    indices_train, indices_test = dataset.get_splits(fold)
    
    train_dl = DataLoader(Subset(dataset, indices_train), batch_size=32)
    test_dl = DataLoader(Subset(dataset, indices_test), batch_size=1024)

    for epoch in range(cfg.num_epochs):

        print("EPOCH:", epoch)
        train_loss = run_train_epoch(cfg, model, train_dl, criterion, optimizer)
        test_loss = run_validation_epoch(cfg, model, test_dl, criterion)

        stopper(test_loss, model)

        message = 'epoch {:d}/{:d}, train {} {:.4f}, \n Test {} {:.4f}, best test {} {:.4f}'.format(
                  epoch + 1, cfg.num_epochs, 'MSE', train_loss,
                  'MSE', test_loss,
                  'MSE', stopper.best_score)

        print(message)

        scheduler.step()

        if stopper.early_stop:
            break

    print("Model saved at " + stopper.path)
    
def get_test(df, fold):
        start_date = "2007-01-"
        end_date = "2012-01-"
        if fold == 1:
            start_date = "2012-01-"
            end_date = "2017-01-"
        elif fold == 2:
            start_date = "2017-01-"
            end_date = "2017-10-"

        start_series = df.date.apply(lambda x: str(x)).str.startswith(start_date)
        end_series = df.date.apply(lambda x: str(x)).str.startswith(end_date)
    
        start_frame = {f"is_January_{start_date[:5]}": start_series}
        end_frame = {f"is_January_{end_date[:5]}": end_series}

        start_df = pd.DataFrame(start_frame)
        end_df = pd.DataFrame(end_frame)

        january_df = start_df.loc[start_df[f'is_January_{start_date[:5]}']==True]
        df_five_years_end = end_df.loc[end_df[f'is_January_{end_date[:5]}']==True]

        start_date_ind = int(list(january_df.index)[-1])
        end_date_ind = int(list(df_five_years_end.index)[-1])
        
        test = [i for i in range(start_date_ind, end_date_ind + 1)]

        return test
    
    
def test_for_a_fold(cfg, fold, dataset, extended = False):
    
    
    indices = get_test(data_pd, fold)
    test_dl = DataLoader(Subset(dataset, indices), batch_size=256)
    if extended:
        model = NeuralNetExtended()
        model.load_state_dict(torch.load(cfg.path(fold, extended=extended)+'/fold' + str(fold) + '_regression.pth'))
    else:
        model = NeuralNet()
        model.load_state_dict(torch.load(cfg.path(fold)+'/fold' + str(fold) + '_regression.pth'))
        
    criterion = nn.MSELoss()
    df, mse = run_validation_epoch(cfg, model, test_dl, criterion, True)
    df.to_csv(cfg.path(fold, extended=extended)+'/preds.csv', index=False)
    return df

Training for 3 folds without differetiating on call and put.

In [137]:
cfg = Config()
cfg.num_epochs = 50
features = ['mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega', 'theta', 'strike', 'delta', 'spot_close'
                   , 'divrate', 'vix', 'short_rate']

dataset = OptionsDataset(cfg.path_to_dataset, features = features)

In [138]:
metrics = {}
for fold in range(3):
    print("Fold", fold)
    train_for_a_fold(cfg, fold, dataset)
    df = test_for_a_fold(cfg, fold, dataset, extended = False)
    mse = mean_squared_error(df['label'], df['predicted'])
    r2 = r2_score(df['label'], df['predicted'])
    metrics[fold] = [mse, r2]
    
metrics

Fold 0


47it [00:00, 444.66it/s]

EPOCH: 0


1273it [00:02, 487.21it/s]
8it [00:00, 51.51it/s]
53it [00:00, 518.46it/s]

Validation loss decreased (inf --> 4.65185070).  Saving model ...
 tensor(-inf, grad_fn=<SubBackward0>)
epoch 1/50, train MSE 413.6057, 
 Test MSE 4.6519, best test MSE 4.6519
EPOCH: 1


...

EarlyStopping counter: 9 out of 10
epoch 10/50, train MSE 668.9487, 
 Test MSE 42.0348, best test MSE 41.9084
EPOCH: 10


4481it [00:09, 462.53it/s]
25it [00:00, 60.71it/s]


EarlyStopping counter: 10 out of 10
epoch 11/50, train MSE 667.7301, 
 Test MSE 42.0808, best test MSE 41.9084
Model saved at /home/sgarkot/code/options/DNN/model/fold_2/fold2_regression.pth


4it [00:00, 169.36it/s]


{0: [44408.168880716374, -0.02357592259128971],
 1: [21088.877123092603, -0.014605315979305145],
 2: [44403.98516768689, -0.031419007403602484]}

In [139]:
metrics

{0: [44408.168880716374, -0.02357592259128971],
 1: [21088.877123092603, -0.014605315979305145],
 2: [44403.98516768689, -0.031419007403602484]}

Adding features

In [140]:
features_2 = []
for f in features:
    if f != 'midprice' and f != 'date':
        features_2.append(f)
        features_2.append(f+"_additional")
total_df.columns

Index(['index', 'mness', 'ttm', 'embed_lev', 'impvol', 'gamma', 'vega',
       'theta', 'strike', 'delta', 'spot_close', 'divrate', 'vix',
       'short_rate', 'midprice', 'date', 'mness_additional', 'ttm_additional',
       'embed_lev_additional', 'impvol_additional', 'gamma_additional',
       'vega_additional', 'theta_additional', 'strike_additional',
       'delta_additional', 'spot_close_additional', 'divrate_additional',
       'vix_additional', 'short_rate_additional'],
      dtype='object')

In [141]:
dataset2 = OptionsDataset('/home/sgarkot/code/options/total_df.csv', csv=True, features = features_2)

metrics = {}
for fold in range(3):
    print("Fold", fold)
    train_for_a_fold(cfg, fold, dataset2, extended = True)
    df = test_for_a_fold(cfg, fold, dataset2, extended = True)
    mse = mean_squared_error(df['label'], df['predicted'])
    r2 = r2_score(df['label'], df['predicted'])
    metrics[fold] = [mse, r2]

metrics


34it [00:00, 336.05it/s]

Fold 0
EPOCH: 0


1273it [00:02, 552.98it/s]
8it [00:00, 88.98it/s]
43it [00:00, 424.66it/s]

Validation loss decreased (inf --> 4.86180401).  Saving model ...
 tensor(-inf, grad_fn=<SubBackward0>)
epoch 1/50, train MSE 414.4681, 
 Test MSE 4.8618, best test MSE 4.8618
EPOCH: 1


1273it [00:02, 537.73it/s]
8it [00:00, 62.33it/s]
36it [00:00, 348.12it/s]

...

EarlyStopping counter: 9 out of 10
epoch 20/50, train MSE 665.4615, 
 Test MSE 41.9454, best test MSE 41.6793
EPOCH: 20


4481it [00:09, 453.43it/s]
25it [00:00, 63.50it/s]


EarlyStopping counter: 10 out of 10
epoch 21/50, train MSE 665.0546, 
 Test MSE 41.8543, best test MSE 41.6793
Model saved at /home/sgarkot/code/options/DNN/model_extended/fold2/fold2_regression.pth


4it [00:00, 161.54it/s]


{0: [45313.987268689896, -0.01394482415826448],
 1: [22265.226988743267, -0.12591023116587285],
 2: [44107.287695653074, -0.02454634321988025]}

In [142]:
metrics

{0: [45313.987268689896, -0.01394482415826448],
 1: [22265.226988743267, -0.12591023116587285],
 2: [44107.287695653074, -0.02454634321988025]}